In [197]:
import random

class SparseMatrix(dict):
    def __init__(self):
        super()
    
    def get_axis(self, axis):
        return [i[axis] for i in self.keys()]
    
    def __repr__(self, default={}):
        rep = ''
        xs = self.get_axis(0) + [0]
        ys = self.get_axis(1) + [0]
        x_range = max(xs) - min(xs)
        y_range = max(ys) - min(ys)
        if max(x_range, y_range) < 20:
            for i in range(min(xs), max(xs) + 1):
                for j in range(min(ys), max(ys) + 1):
                    rep += self.get((i,j),default.get((i,j),'.'))
                rep += '\n'
        return rep

In [196]:
class Bag:
    def __init__(self):
        letters = 'qwertyuiopasdfghjklzxcvbnm'
        self.letters = letters
    
    def draw(self, n=1):
        pass
    
class Dictionary:
    def __init__(self):
        pass
    def is_word(self, x):
        return x in ['hello','world','ab']
    
class Player:
    def __init__(self, name):
        self.name = name
        self.rack = ''
    
class Players:
    def __init__(self, players):
        self.players = [Player(p) for p in players]
        self.turn = 0
    
    def current_player(self):
        return self.players[self.turn]
    
    def next_turn(self):
        self.turn += 1
        self.turn %= len(self.players)
        return self.current_player()

class Game:
    def __init__(self, players=['P1','P2']):
        self.size = 10
        self.board = {}
        self.bonuses = {(1,1): 'dw',(1,2): 'dl'}
        self.bag = Bag()
        self.dictionary = Dictionary()
        self.players = Players(players)
        self.points = {
            'a': 1,
            'b': 1,
            'c': 1,
            'h': 1,
        }
    
    def parse_bonus(self, bonus):
        if bonus == 'dl':
            return 2,1
        if bonus == 'tl':
            return 3,1
        if bonus == 'dw':
            return 1,2
        if bonus == 'tw':
            return 1,3
        return 1,1
            
    def scan_move(self, start_coord, direction, move):
        x,y = start_coord
        Directions = {'h':[1,0],'v':[0,1]}
        dx, dy = Directions.get(direction, direction)
        # Go to the first letter of the word given the direction
        while (x-dx, y-dy) in self.board or (x-dx, y-dy) in move:
            x -= dx
            y -= dy
        word = ''
        score = 0
        total_word_scale = 1
        scanned_coords = []
        num_tiles_from_move = 0
        # Now read through the word in order
        p = (x,y)
        while p in self.board or p in move:
            # Check tile is not already filled
            if p in self.board and p in move:
                print(f'Tried to play {move[p]} at {p}')
                print(f'Spot already filled by {self.board[p]} at {p}')
                assert False
            if p in self.board:
                letter = self.board[p]
                word += letter
                score += self.points.get(letter,0)
            if p in move:
                letter = move[p]
                word += letter
                bonus = self.bonuses.get(p)
                letter_scale, word_scale = self.parse_bonus(bonus)
                score += letter_scale * self.points.get(letter, 0)
                total_word_scale *= word_scale
                num_tiles_from_move += 1
            scanned_coords.append(p)
            x,y = p
            x += dx
            y += dy
            p = (x,y)
        score *= total_word_scale
        print(f'found {num_tiles_from_move} tile from move')
        print(f'found {word} for {score} points')
        return word, scanned_coords, score
        
    def play(self, player, move):
        # Check it is the players turn
        assert player == self.players.current_player().name
        # Check player has the letters in their rack
#         for letter in move.values():
#             assert letter in self.players.current_player().rack
            
        xs = [i[0] for i in move.keys()]
        ys = [i[1] for i in move.keys()]
        # Check played tiles are on the board bounds
        assert all(x >= 0 for x in xs)
        assert all(x < self.size for x in xs)
        assert all(y >= 0 for y in ys)
        assert all(y < self.size for y in ys)
        
        # Find words for each played tile
        scanned = {'h': {}, 'v':{}}
        times_scanned = {'h':0,'v':0}
        words = []
        scores = []
        for pos in move:
            for d in scanned.keys(): # for direction in ['h','v']
                if pos not in scanned[d]:
                    print(f'Scanning {pos} in direction: {d}')
                    times_scanned[d] += 1
                    word, scanned_coords, score = self.scan_move(pos, d, move)
                    for s in scanned_coords:
                        scanned[d][s] = True
                    if len(word) > 1:
                        words.append(word)
                        scores.append(score)
        print('found',words,'for',scores,'points')
        print(times_scanned)
        # Check word is played in a continuous line
        assert 1 in times_scanned.values()
        
        # Check at least 1 word was found
        assert len(words) > 0
        # Check each word is a real word
        for word in words:
            assert self.dictionary.is_word(word)
            
        # Find score of word
        # Place tiles on board
        # remove from player rack
        # advance to the next player
        self.players.next_turn()
                    
game = Game()
game.play('P1', {(1,1):'a', (1,2):'b'})

Scanning (1, 1) in direction: h
found 1 tile from move
found a for 2 points
Scanning (1, 1) in direction: v
found 2 tile from move
found ab for 6 points
Scanning (1, 2) in direction: h
found 1 tile from move
found b for 2 points
found ['ab'] for [6] points
{'h': 2, 'v': 1}


In [185]:
valid_move = {
    (1,1): 'h',
    (1,2): 'e',
    (1,3): 'l',
    (1,4): 'l',
    (1,5): 'o',
}
invalid_move = {
    (1,1): 'h',
    (1,2): 'e',
    (1,3): 'l',
    (1,4): 'l',
    (1,6): 'o',
}

In [186]:
game.play('P2',valid_move)

Scanning (1, 1) in direction: h
found 1 tile from move
found h for 20 points
Scanning (1, 1) in direction: v
found 5 tile from move
found hello for 20 points
Scanning (1, 2) in direction: h
found 1 tile from move
found e for 0 points
Scanning (1, 3) in direction: h
found 1 tile from move
found l for 0 points
Scanning (1, 4) in direction: h
found 1 tile from move
found l for 0 points
Scanning (1, 5) in direction: h
found 1 tile from move
found o for 0 points
found ['hello'] for [20] points
{'h': 5, 'v': 1}


In [177]:
game.play('P1',invalid_move)

Scanning (1, 1) in direction: h
found 1 tile from move
Scanning (1, 1) in direction: v
found 4 tile from move
Scanning (1, 2) in direction: h
found 1 tile from move
Scanning (1, 3) in direction: h
found 1 tile from move
Scanning (1, 4) in direction: h
found 1 tile from move
Scanning (1, 6) in direction: h
found 1 tile from move
Scanning (1, 6) in direction: v
found 1 tile from move
found ['hell']
{'h': 5, 'v': 2}


AssertionError: 